# import

In [40]:
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt
from my_library.library import *
from pprint import pprint
import json
import requests
import sys
path_ =  '/Users/Owner/Desktop/coincheck_api'
sys.path.append(path_)
from settings_coincheck import API_settings as capi
import ccxt
import datetime
import logging
from pprint import pprint
import json
import requests
import time

In [6]:
API_KEY = capi.API_KEY
API_SECRET = capi.API_SECRET
SYMBOL = "BTC/JPY"

coincheck = ccxt.coincheck({'apiKey':API_KEY,'secret':API_SECRET})

In [ ]:
#　取引通貨情報
def market_info():
    result = coincheck.load_markets()
    return result

pprint(market_info())

In [30]:
def ticker_info(SYMBOL):
    result = coincheck.fetch_ticker(symbol=SYMBOL)
    return result

pprint(ticker_info(SYMBOL))
board = ticker_info(SYMBOL)
ask = board['ask']
bid = board['bid']

{'ask': 2831300.0,
 'askVolume': None,
 'average': None,
 'baseVolume': 2764.66301029,
 'bid': 2830379.0,
 'bidVolume': None,
 'change': None,
 'close': 2831300.0,
 'datetime': '2022-08-30T07:58:43.000Z',
 'high': 2853303.0,
 'info': {'ask': '2831300.0',
          'bid': '2830379.0',
          'high': '2853303.0',
          'last': '2831300.0',
          'low': '2740376.0',
          'timestamp': '1661846323',
          'volume': '2764.66301029'},
 'last': 2831300.0,
 'low': 2740376.0,
 'open': None,
 'percentage': None,
 'previousClose': None,
 'quoteVolume': None,
 'symbol': 'BTC/JPY',
 'timestamp': 1661846323000,
 'vwap': None}


In [38]:
time

Wall time: 0 ns


In [44]:
while True:
    board = ticker_info(SYMBOL)
    ask = board['ask']
    bid = board['bid']
    spread = (ask - bid)/bid
    print("-------------------")
    print("ask",ask)
    print("bid",bid)
    print("spread",spread)
    time.sleep(2)

-------------------
ask 2830498.0
bid 2830001.0
spread 0.00017561831250236307
-------------------
ask 2830498.0
bid 2830001.0
spread 0.00017561831250236307
-------------------
ask 2830498.0
bid 2830001.0
spread 0.00017561831250236307
-------------------
ask 2830498.0
bid 2830001.0
spread 0.00017561831250236307
-------------------
ask 2829943.0
bid 2829241.0
spread 0.00024812308318732833
-------------------
ask 2829943.0
bid 2829241.0
spread 0.00024812308318732833
-------------------
ask 2829943.0
bid 2829241.0
spread 0.00024812308318732833
-------------------
ask 2829943.0
bid 2829241.0
spread 0.00024812308318732833
-------------------
ask 2829943.0
bid 2829241.0
spread 0.00024812308318732833
-------------------
ask 2829943.0
bid 2829241.0
spread 0.00024812308318732833
-------------------
ask 2829694.0
bid 2829240.0
spread 0.00016046712191259845


KeyboardInterrupt: 

In [123]:
def balance_info():
    result = coincheck.fetchBalance()
    return result


pprint(balance_info())

{'BTC': {'free': 0.005, 'total': 0.005, 'used': 0.0},
 'ETC': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'JPY': {'free': 35936.56, 'total': 35936.56, 'used': 0.0},
 'MONA': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'free': {'BTC': 0.005, 'ETC': 0.0, 'JPY': 35936.56, 'MONA': 0.0},
 'info': {'bat': '0.0',
          'bat_debt': '0.0',
          'bat_lend_in_use': '0.0',
          'bat_lent': '0.0',
          'bat_reserved': '0.0',
          'bch': '0.0',
          'bch_debt': '0.0',
          'bch_lend_in_use': '0.0',
          'bch_lent': '0.0',
          'bch_reserved': '0.0',
          'btc': '0.005',
          'btc_debt': '0.0',
          'btc_lend_in_use': '0.0',
          'btc_lent': '0.0',
          'btc_reserved': '0.0',
          'enj': '0.0',
          'enj_debt': '0.0',
          'enj_lend_in_use': '0.0',
          'enj_lent': '0.0',
          'enj_reserved': '0.0',
          'etc': '0.0',
          'etc_debt': '0.0',
          'etc_lend_in_use': '0.0',
          'etc_lent': '

In [64]:
ask

2829694.0

In [65]:
bid

2829240.0

# 売買　コード

In [67]:
board

{'symbol': 'BTC/JPY',
 'timestamp': 1661848233000,
 'datetime': '2022-08-30T08:30:33.000Z',
 'high': 2853303.0,
 'low': 2740376.0,
 'bid': 2825630.0,
 'bidVolume': None,
 'ask': 2826413.0,
 'askVolume': None,
 'vwap': None,
 'open': None,
 'close': 2826250.0,
 'last': 2826250.0,
 'previousClose': None,
 'change': None,
 'percentage': None,
 'average': None,
 'baseVolume': 2761.97413964,
 'quoteVolume': None,
 'info': {'last': '2826250.0',
  'bid': '2825630.0',
  'ask': '2826413.0',
  'high': '2853303.0',
  'low': '2740376.0',
  'volume': '2761.97413964',
  'timestamp': '1661848233'}}

In [72]:
# 引数情報
board = ticker_info(SYMBOL)

order_symbol = 'BTC/JPY'   # 取引通貨
type_value   = 'limit'     # 注文タイプ（limit:指値注文, market:成行注文）
side_value   = 'sell'       # 買い(buy) or 売り(sell)
price_value  = board['bid'] - 1     # 指値価格[円/BTC]
amount_value = 0.005       # 取引数量[BTC]


# 注文コード
order = coincheck.create_order(
                symbol = order_symbol,  # 取引通貨
                type = type_value,      # 注文タイプ
                side= side_value,       # 買 or 売
                amount= amount_value,   # 取引数量
                price=price_value,      # 指値価格
                )

# 結果出力
pprint(order)

{'id': '4942415462',
 'info': {'amount': '0.005',
          'created_at': '2022-08-30T08:34:15.000Z',
          'id': '4942415462',
          'market_buy_amount': None,
          'order_type': 'sell',
          'pair': 'btc_jpy',
          'rate': '2826116.0',
          'stop_loss_rate': None,
          'success': True}}


In [24]:
order['id']

'4942326801'

In [71]:
def OrderCancel(order_id, symbol):
    result = coincheck.cancel_order(
                    symbol = symbol,  # 取引通貨
                    id = order_id,    # 注文ID
                    )
    return result

# 引数情報
order_id = order['id']
symbol =   'BTC/JPY'

# 出力
pprint(OrderCancel(order_id,symbol))

{'id': '4942404971', 'success': True}


# 未約定のorderの注文照会


In [81]:
OpenOrder(symbol) == []

True

In [84]:
def OpenOrder(symbol):
    result = coincheck.fetchOpenOrders(symbol = symbol)
    return result

# 引数情報
symbol = 'BTC/JPY'

# 出力
pprint(OpenOrder(symbol))

[{'amount': 0.005,
  'average': None,
  'clientOrderId': None,
  'cost': 0.0,
  'datetime': '2022-08-30T09:21:07.000Z',
  'fee': None,
  'fees': [],
  'filled': 0.0,
  'id': '4942503271',
  'info': {'created_at': '2022-08-30T09:21:07.000Z',
           'id': '4942503271',
           'order_type': 'buy',
           'pair': 'btc_jpy',
           'pending_amount': '0.005',
           'pending_market_buy_amount': None,
           'rate': '2823383.0',
           'stop_loss_rate': None},
  'lastTradeTimestamp': None,
  'postOnly': None,
  'price': 2823383.0,
  'remaining': 0.005,
  'side': 'buy',
  'status': 'open',
  'stopPrice': None,
  'symbol': 'BTC/JPY',
  'timeInForce': None,
  'timestamp': 1661851267000,
  'trades': [],
  'type': None}]


In [87]:
def Yakujo(symbol):
    result = coincheck.fetchMyTrades(symbol = symbol)
    return result

# 引数情報
symbol = 'BTC/JPY'

# 出力
pprint(Yakujo(symbol))

[]


In [76]:
def buy(price_value):
    order_symbol = 'BTC/JPY'   # 取引通貨
    type_value   = 'limit'     # 注文タイプ（limit:指値注文, market:成行注文）
    side_value   = 'buy'       # 買い(buy) or 売り(sell)
    amount_value = 0.005       # 取引数量[BTC]


    # 注文コード
    order = coincheck.create_order(
                    symbol = order_symbol,  # 取引通貨
                    type = type_value,      # 注文タイプ
                    side= side_value,       # 買 or 売
                    amount= amount_value,   # 取引数量
                    price=price_value,      # 指値価格
                    )
    return  order

def sell(price_value):
    order_symbol = 'BTC/JPY'   # 取引通貨
    type_value   = 'limit'     # 注文タイプ（limit:指値注文, market:成行注文）
    side_value   = 'sell'       # 買い(buy) or 売り(sell)
    amount_value = 0.005       # 取引数量[BTC]


    # 注文コード
    order = coincheck.create_order(
                    symbol = order_symbol,  # 取引通貨
                    type = type_value,      # 注文タイプ
                    side= side_value,       # 買 or 売
                    amount= amount_value,   # 取引数量
                    price=price_value,      # 指値価格
                    )
    return  order
    




In [93]:
symbol = 'BTC/JPY'
board = ticker_info(symbol)

buy_price = board['bid'] + 1
sell_price = board['ask'] - 1

order_buy = buy(buy_price)

In [107]:
order_now = OpenOrder(symbol)
order_now


[]

In [109]:
len([])

0

In [ ]:

order_now = OpenOrder(symbol)

if len(order_now)==0:
    is_buy = True
    is_sell = True
elif len(order_now)==1:
    side = order_now[0]['side']
    if side == 'buy':
        is_sell = True
    elif side == 'sell':
        is_buy = True
elif len(order_now)==2:
    pass

In [111]:
try:
    order_id = order_buy['id']
    order_cancel = OrderCancel(order_id,symbol)
except Exception as e:
    print(e)

# キャンセルできないときは売れたと仮定
# OpenOrder(symbol)

coincheck {"success":false,"error":"Failed to cancel the order."}


In [97]:
coincheck.fetch_my_trades(symbol)

[]

In [134]:
order_now = OpenOrder(symbol)
order_now

[]

In [175]:
board = ticker_info(symbol)
buy_price = board['bid'] + 1
sell_price = board['ask'] - 1
order_sell = sell(sell_price)
# order_buy = buy(buy_price)

In [176]:
pprint(balance_info()['BTC']['free'])
pprint(balance_info()['JPY'])

0.005
{'free': 35942.42, 'total': 35942.42, 'used': 0.0}


In [172]:
profit

5226.0

In [171]:
loop_count

0

In [170]:
trade_count

12

In [135]:
def get_status(order_now):
    is_buy = False
    is_sell = False    
    if len(order_now)==0:
        is_buy = True
        is_sell = True
    elif len(order_now)==1:
        side = order_now[0]['side']
        if side == 'buy':
            is_sell = True
        elif side == 'sell':
            is_buy = True
    elif len(order_now)==2:
        pass
    
    return is_buy, is_sell

In [168]:
order_now = OpenOrder(symbol)
order_now

[]

In [166]:
symbol = 'BTC/JPY'

is_buy = False
is_sell = False
loop_count = 0
trade_count = 0
profit = 0
is_trade = False
board = ticker_info(symbol)
buy_price = board['bid'] + 1
sell_price = board['ask'] - 1

while True:


    
    if not is_buy and not is_sell and not is_trade:
        order_buy = buy(buy_price)
        order_sell = sell(sell_price)
        is_trade = True
        time.sleep(5)

    order_now = OpenOrder(symbol)
    is_buy, is_sell = get_status(order_now)



    if is_buy and is_sell:
        is_buy = False
        is_sell = False
        trade_count += 1
        profit += sell_price - buy_price
        is_trade = False

    elif not is_buy and is_sell:
        board = ticker_info(symbol)
        buy_price = board['bid'] + 1
        try:
            order_id = order_buy['id']
            order_cancel = OrderCancel(order_id,symbol)
            order_buy = buy(buy_price)
            time.sleep(1)
            
        except Exception as e:
            order_now = OpenOrder(symbol)
            is_buy, is_sell = get_status(order_now)
            print(1)
            print(e)

    elif is_buy and not is_sell:
        board = ticker_info(symbol)
        sell_price = board['ask'] - 1
        try:
            order_id = order_sell['id']
            order_cancel = OrderCancel(order_id,symbol)
            order_sell = sell(sell_price)
            time.sleep(1)
        except Exception as e:
            order_now = OpenOrder(symbol)
            is_buy, is_sell = get_status(order_now)
            print(2)
            print(e)
        

    elif not is_buy and not is_sell:
        time.sleep(5)
        order_now = OpenOrder(symbol)
        is_buy, is_sell = get_status(order_now)
        # try:
        #     order_id = order_sell['id']
        #     order_cancel = OrderCancel(order_id,symbol)
        #     is_sell = False
            
        #     order_id = order_buy['id']
        #     order_cancel = OrderCancel(order_id,symbol)
        #     is_buy = False
            
        # except Exception as e:
        #     order_now = OpenOrder(symbol)
        #     is_buy, is_sell = get_status(order_now)
        #     loop_count += 1
        #     print(e)





1
coincheck {"success":false,"error":"Failed to cancel the order."}
1
coincheck {"success":false,"error":"Failed to cancel the order."}


KeyboardInterrupt: 

In [120]:
trade_count

0

In [102]:
board = ticker_info(symbol)
buy_price = board['bid'] + 1
sell_price = board['ask'] - 1

order_buy = buy(buy_price)
order_sell = sell(sell_price)